In [2]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime
import plotly.offline as offline 
import plotly.graph_objs as go
from plotly import subplots

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
code_df = code_df[['회사명', '종목코드']]
 
# 한글로된 컬럼명을 영어로 바꿔준다.
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
# code_df.head()
tmp = code_df.query('name=="대한한공"')
tmp['code'].to_string(index=False)

c:\users\administrator\appdata\local\programs\python\python37-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


'Series([], )'

In [17]:
def get_code(item_name, code_df):
    return code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()






# 일자(n,m,t)에 따른 Stochastic(KDJ)의 값을 구하기 위해 함수형태로 만듬 
def get_stochastic(df, n=15, m=5, t=3):
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌
    df = pd.DataFrame(df)
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j)
    
    return df



def start(item_name, code_df):
    start = datetime(2018,1,1)
    end = datetime(2021,1,1)
    
    code = get_code(item_name, code_df)

    df = data.get_data_yahoo('{}.KS'.format(code), start, end)
    df = df.rename_axis('Date').reset_index()
    print(df.tail())
    df = get_stochastic(df)


 
    # jupyter notebook 에서 출력 
    offline.init_notebook_mode(connected=True)
 
    kdj_k = go.Scatter(
                x=df.Date,
                y=df['kdj_k'],
                name="Fast%K")
 
    kdj_d = go.Scatter(
                x=df.Date,
                y=df['kdj_d'],
                name="Fast%D")
 
    kdj_d2 = go.Scatter(
                x=df.Date,
                y=df['kdj_d'],
                name="Slow%K")
 
    kdj_j = go.Scatter(
                x=df.Date,
                y=df['kdj_j'],
                name="Slow%D")
 
    lines = go.Scatter(
                x=df.Date,
                y=df['Close'],
                mode='lines',
                line=dict(color='black',width=2),
                name="line")
    trade_volume = go.Bar(
                x=df.Date,
                y=df['Volume'],
                name="Volume")
 

    price = go.Candlestick(
                x=df.Date,
                open=df['Open'],
                high=df['Open'],
                low=df['Low'],
                close=df['Close'])
    

    data1 = [kdj_d2, kdj_j]
    data2 = [trade_volume]
    data3 = [trade_volume]
 
    fig = subplots.make_subplots(rows=3, cols=1, shared_xaxes=True)
 
    for trace in data1:
        fig.append_trace(trace, 1,1)
 
    for trace in data2:
        fig.append_trace(trace, 2,1)
    


    fig.update_layout(height=900, width=1030)

    
    fig2 = go.Figure(data=price)
    fig2.show()
    fig.show()
    
start('대한항공', code_df)




          Date     High      Low     Open    Close     Volume  Adj Close
481 2020-02-13  26550.0  25900.0  26100.0  26050.0   744099.0    26050.0
482 2020-02-14  26150.0  25600.0  26050.0  25650.0   425300.0    25650.0
483 2020-02-17  25650.0  25100.0  25550.0  25300.0   462173.0    25300.0
484 2020-02-18  25250.0  24400.0  25250.0  24450.0   891527.0    24450.0
485 2020-02-24  22850.0  22050.0  22800.0  22100.0  1619042.0    22100.0
